# Evaluating SETI detection performance with swept parameters
This notebook runs multiple instances of notebook "01_seti_end_to_end.ipynb", which does the following:
1) Inputs a single RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)

The primary sweep parameters are 
1) FFT size (which determines frequency resolution and equivalent noise bandwidth)
2) Short Term Integration (Pre-DeDoppler) spectrogram averaging parameter n_sti (aka int_factor or N_preDD)

Of special interest is a RAW file with multiple simultaneous drifting signals.  Parameters for GBT, MeerKAT, or VLASS may be selected.

Note that these sweeps can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time
import numpy as np
import array
%matplotlib inline
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'medium',
          'figure.figsize': (10,6),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

import src.plot_fns as pltg             # generic plot fns
import src.plot_h5_psd_sg1 as plt_h5    # blimpy-based plot fns

parameters_are_undefined = False

verbose = False
fb_ext = '.h5'

t_turbo = np.NaN

search_max_drift = 10
search_min_drift = -search_max_drift

drift_limit_nHz = 1.0  # assume 2 sigma limit
f_sigma_drift = np.array([1.5, 3, 6])*1e9
sigma_drift = drift_limit_nHz*f_sigma_drift/2.*1e-9
print(f'sigma_drift = {sigma_drift}')

raw_dir = os.getenv('RAWDIR') + '/'
if not os.path.isdir(raw_dir[0:-1]):
    os.system('mkdir '+raw_dir[0:-1])

raw_backup_base_dir = os.getenv('RAW_BACKUP_BASE_DIR') + '/'

def file_name_mjd(raw_file_stem):
    # specific to guppi file naming conventions
    stem_parts = raw_file_stem.split('_')
    if (len(stem_parts)<4):
         mjd_int = 99999
    elif (stem_parts[1]=='guppi'):
        mjd_int = int(stem_parts[2])
    elif (stem_parts[2]=='guppi'):
        mjd_int = int(stem_parts[3])
    else:
        mjd_int = 99999
    #print(mjd_int)   
    return mjd_int


In [ ]:
try:
    raw_files_are_undefined
except NameError:
    raw_files_are_undefined = True     
    print('Raw files are undefined, using defaults\n')

if raw_files_are_undefined:
    display_figs01 = False
    display_figs02 = True
    plot_dets1 = True
    plot_dets2 = True
    zoom_bw_MHz = .1
    plot_spectra = False
    plot_snr_vs_f = False
    do_profile = False
    do_fig_text_only = True   # True normally, False to generate copy of figures without text for pubs
    
    if (1):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec'
    elif (0):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec'
    elif (1):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec'
    elif (1):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'cosmic-chirp80-0.0040V-1401.70-1402.30-m10.0-10.0-Hzsec-67.11sec'
    elif (1):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'cosmic-chirp80-0.0040V-1401.70-1402.30-m50.0-50.0-Hzsec-67.11sec'
        search_max_drift = 50
        search_min_drift = -50
    elif (1):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec'
    elif (0):
        raw_backup_subdir = 'raw_voyager/'
        raw_file_stem = 'blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015' # 23.7 dB, -.504 Hz/sec
    elif(1):
        raw_backup_subdir = 'raw_gbt_59103_HIP/'
        raw_file_stem = 'blc40_guppi_59103_03394_DIAG_HIP95631_0015' # 23.7 dB, -.504 Hz/sec
        
else:
    display_figs01 = False
    display_figs02 = False
    do_fig_text_only = True

                
print(f'{raw_file_stem = }')

delete_raw_file = False
run_turbo = False

if do_fig_text_only:
    n_fig_copies = 1
else:
    n_fig_copies = 2    # second copy created without text in plot area


In [ ]:
delete_sg_file = True

raw_file_base_name = raw_dir + raw_file_stem + '.0000.raw'
raw_file_delete_spec = raw_dir + raw_file_stem + '.00*.raw'

print(raw_file_base_name)
if os.path.isfile(raw_file_base_name): 
    print('Raw file found in raw directory')
else:
    raw_file_backup_name = raw_backup_base_dir + raw_backup_subdir + raw_file_stem + '.0000.raw'
    raw_file_backup_spec = raw_backup_base_dir + raw_backup_subdir + raw_file_stem + '.00*.raw'
    if os.path.isfile(raw_file_backup_name): 
        print('Raw file found in backup directory, copying')
        if (1):     # single raw file .0000.raw
            print(raw_file_backup_name)
            os.system('cp '+raw_file_backup_name+' '+raw_dir)
            os.system('chmod 666 '+raw_dir+'*.raw')  # remove read-only spec from raw_dir
        else:       # multi raw file .00xx.raw
            os.system('ls '+raw_file_backup_spec)
            os.system('cp '+raw_file_backup_spec+' '+raw_dir)
            os.system('chmod 666 '+raw_dir+'*.raw')  # remove read-only spec from raw_dir
        os.system('ls -lsah '+raw_dir+'*.raw')
        print('Copy complete')
        delete_raw_file = True
    else:
        print('Raw file not found in backup directory, skip')
        print(raw_file_backup_name)
        # stop, or return to calling notebook/script
        assert(False)


#### Run end-to-end test over parameter sweeps

In [ ]:
isChirp = raw_file_stem.lower().find('chirp')>=0
isVoyager = raw_file_stem.lower().find('voyager')>=0
isguppi = raw_file_stem.lower().find('guppi')>=0

if isChirp:
    raw_parts_list = raw_file_stem.split('-')
    telescope = raw_parts_list[0]

    if (raw_file_stem.lower().find('gbt')>=0):    # GBT/Parkes HSR2A, but 512 avg rather than 51*16=816 avg (183 sec vs 292)
    
        fine_fft_size_list = np.array([1, 2, 4])*1024*512
        # n_sti_list = np.array([1, 2, 4, 8, 16, 32])
        n_sti_list = np.array([2, 4, 8, 16, 32, 64])
        select_params = [[1024*1024,32],[1024*1024,4],[1024*1024*2,2]]
        select_params2 =  [[512*1024, 2], [1024*1024, 2], [2048*1024, 2]]
        fig_dir = './chirp_gbt_plots/'

    elif(raw_file_stem.lower().find('meerkat')>=0):   # MeerKAT 1K mode
    
        fine_fft_size_list = np.array([1, 2, 4, 8])*1024*256
        n_sti_list = np.array([1, 2, 4, 8, 16])
        select_params = [[1024*512,8],[1024*512,2],[1024*512,1],[1024*1024,1]]
        select_params2 =  [[256*1024, 4],[512*1024, 2], [1024*1024, 1], [2048*1024, 1]]
        fig_dir = './chirp_mk_plots/'
    
    elif(raw_file_stem.lower().find('cosmic')>=0):  # COSMIC VLASS
    
        fine_fft_size_list = np.array([1, 2, 4])*1024*256
        n_sti_list = np.array([1, 2, 4])
        select_params = [[1024*512,1],[1024*512,2],[1024*256,1]]
        select_params2 =  [[256*1024, 1], [512*1024, 1], [1024*1024, 1]]
        fig_dir = './chirp_cosmic_plots/'
    
    elif(raw_file_stem.lower().find('vlass')>=0):  # COSMIC VLASS
    
        fine_fft_size_list = np.array([1, 2])*1024*128
        n_sti_list = np.array([1, 2, 4])
        select_params = [[1024*128,1],[1024*256,2],[1024*256,1]]
        select_params2 =  [[128*1024, 1], [256*1024, 1]]
        fig_dir = './chirp_vlass_plots/'

    plot_snr_vs_f_zoom = False
    zoom_bw_MHz = .1
    plot_dets1 = True
    plot_dets2 = True

elif (isguppi):  #
 
    print(f'Raw file {raw_file_stem}, {isguppi = }')
    mjd_int = file_name_mjd(raw_file_stem)
    if (mjd_int > 58000):
        if (0):
            fine_fft_size_list = np.array([1, 2])*1024*1024
            n_sti_list = np.array([1, 2, 4, 8])
        else:
            fine_fft_size_list = np.array([1])*1024*1024
            n_sti_list = np.array([4,])
    else:
        if (0):
            fine_fft_size_list = np.array([1, 2])*1024*1009
            n_sti_list = np.array([1, 2, 4, 8])
        else:
            fine_fft_size_list = np.array([1])*1024*1009
            n_sti_list = np.array([4,])
    
    
    select_params = []
        
    search_max_drift = 10
    search_min_drift = -10
    plot_snr_vs_f_zoom = True
    plot_dets1 = True
    plot_dets2 = True

    if isVoyager:
        fig_dir = './voyager_plots/'
    else:
        fig_dir = f'./{raw_backup_subdir[0:-1]}_plots/'

print(f'Figures will appear in in {fig_dir}')


Determine duration of raw file and other parameters

In [ ]:
fine_fft_size = fine_fft_size_list[0]
n_sti = 1
import src.get_raw_info as raw
p = raw.get_run_params(raw_file_stem,raw_dir,fine_fft_size,n_sti)
# copy p entries into local variables
%run -i "src/raw_info_script.py"

fft_avg_product = fine_fft_size*n_sti*n_lti


#### Run 01_seti_end_to_end.ipynb for range of fft sizes and sti averages

In [ ]:

i_run = 0
n_fft_list = len(fine_fft_size_list)
len_n_sti_list = len(n_sti_list)
n_run = n_fft_list*len_n_sti_list

n_det_max = 2000
n_sigma_drift = len(sigma_drift)

import time
import numpy as np

time_rawspec = np.ones((n_fft_list,len_n_sti_list))*np.nan
time_turbo = np.ones((n_fft_list,len_n_sti_list))*np.nan
time_turbo_total = np.ones((n_fft_list,len_n_sti_list))*np.nan
time_core = np.ones((n_fft_list,len_n_sti_list))*np.nan
time_core_total = np.ones((n_fft_list,len_n_sti_list))*np.nan
time_run = np.ones((n_fft_list,len_n_sti_list))*np.nan

f_start_truth_all = np.ones((n_det_max,n_fft_list,len_n_sti_list))*np.nan
drift_rate_truth_all = np.ones((n_det_max,n_fft_list,len_n_sti_list))*np.nan
ref_snr_db_all = np.ones((n_det_max,n_fft_list,len_n_sti_list))*np.nan

n_det_core = np.ones((n_fft_list,len_n_sti_list))*np.nan
f_start_core_all = np.ones((n_det_max,n_fft_list,len_n_sti_list))*np.nan
drift_rate_core_all = np.ones((n_det_max,n_fft_list,len_n_sti_list))*np.nan
snr_core_db_all = np.ones((n_det_max,n_fft_list,len_n_sti_list))*np.nan

snr_core_db_wavg_all = np.ones((n_fft_list,len_n_sti_list,n_sigma_drift))*np.nan

peak_snr_all = np.ones((n_fft_list,len_n_sti_list))*np.nan
f_start_max_MHz_all = np.ones((n_fft_list,len_n_sti_list))*np.nan
drift_rate_max_all = np.ones((n_fft_list,len_n_sti_list))*np.nan

h5_size_MB_all = np.zeros((n_fft_list,len_n_sti_list))

n_lti_list = np.ones((n_fft_list,len_n_sti_list))*np.nan
T_avg_list = np.ones((n_fft_list,len_n_sti_list))*np.nan

print('fine_fft_size_list=',fine_fft_size_list)
print('n_sti_list=',n_sti_list,'\n')

import os
# open directory for figures if necessary
if not os.path.isdir(fig_dir[0:-1]):
    os.system('mkdir '+fig_dir[0:-1])

for i_fft, fine_fft_size in enumerate(fine_fft_size_list):
    for i_sti, n_sti in enumerate(n_sti_list):
        
        p = raw.get_run_params(raw_file_stem,raw_dir,fine_fft_size,n_sti)
        # copy p entries into local variables
        %run -i "src/raw_info_script.py"
    
        i_run += 1
        print(f'\nRun {i_run} of {n_run}')
        print(f'{fine_fft_size=}, {n_sti=}, {n_lti=}\n')

        if (n_lti<=2):
            print(f'Note: {n_lti=}, skipping this case\n')
            n_lti_list[i_fft,i_sti] = -n_lti
            continue
        
        t_run_start = time.time()
        
        # try:

        %run ./01_seti_end_to_end.ipynb
    
        # Note times are per raw channel
        time_rawspec[i_fft,i_sti] = t_rawspec
        time_turbo[i_fft,i_sti] = t_turbo
        time_core[i_fft,i_sti] = t_core
        time_turbo_total[i_fft,i_sti] = t_rawspec + t_turbo
        time_core_total[i_fft,i_sti] = t_rawspec + t_core
        time_run[i_fft,i_sti] = time.time() - t_run_start

        f_start_truth_all[0:n_chirp,i_fft,i_sti] = f_start_truth
        drift_rate_truth_all[0:n_chirp,i_fft,i_sti] = drift_rate_truth
        ref_snr_db_all[0:n_chirp,i_fft,i_sti] = ref_snr_db

        n_det1 = min(n_det_max,len(snr_core_db))
        n_det_core[i_fft,i_sti] = n_det1
    
        f_start_core_all[0:n_det1,i_fft,i_sti] = f_start_core[0:n_det1]
        drift_rate_core_all[0:n_det1,i_fft,i_sti] = drift_rate_core[0:n_det1]
        snr_core_db_all[0:n_det1,i_fft,i_sti] = snr_core_db[0:n_det1]

        snr_core_db_wavg_all[i_fft,i_sti,0:n_sigma_drift] = snr_core_db_wavg

        peak_snr_all[i_fft,i_sti] = snr_max
        f_start_max_MHz_all[i_fft,i_sti] = f_start_max_MHz
        drift_rate_max_all[i_fft,i_sti] = drift_rate_max

        h5_size_MB_all[i_fft,i_sti] = h5_size_MB
        n_lti_list[i_fft,i_sti] = n_lti
        T_avg_list[i_fft,i_sti] = p['t_obs']
        
        # except:
        #     n_lti_list[i_fft,i_sti] = -n_lti
        #     print(f'{n_sti=}, {n_lti=}, Error occurred in this case\n\n')


#### Store results in npz file for later recall

In [ ]:
# Write the data to a np savez file
z_name = fig_dir + raw_file_stem + '-' + test_case + ".npz"
print('Writing results to file',z_name)
np.savez_compressed(z_name,test_case=test_case,fine_fft_size_list=fine_fft_size_list,
                    n_sti_list=n_sti_list,n_lti_list=n_lti_list,T_avg_list=T_avg_list,telescope=telescope,
                    samples_per_block=samples_per_block,search_max_drift=search_max_drift,
                    search_min_drift=search_min_drift,search_z_threshold=search_z_threshold,
                    select_params=select_params,select_params2=select_params2,
                    f_sigma_drift=f_sigma_drift,sigma_drift=sigma_drift,
                    time_rawspec=time_rawspec,time_turbo=time_turbo,
                    time_core=time_core,time_turbo_total=time_turbo_total,
                    time_core_total=time_core_total,time_run=time_run,
                    f_start_truth_all=f_start_truth_all,drift_rate_truth_all=drift_rate_truth_all,
                    ref_snr_db_all=ref_snr_db_all,n_det_core=n_det_core,
                    n_det1=n_det1,f_start_core_all=f_start_core_all,
                    drift_rate_core_all=drift_rate_core_all,snr_core_db_all=snr_core_db_all,
                    snr_core_db_wavg_all=snr_core_db_wavg_all,peak_snr_all=peak_snr_all,
                    f_start_max_MHz_all=f_start_max_MHz_all,drift_rate_max_all=drift_rate_max_all,
                    h5_size_MB_all=h5_size_MB_all)

#### Plot composite results: SNR vs Drift Rate

Note: test_case string is defined within 01_seti_end_to_end

In [ ]:
if isChirp:
    for i_fft, fine_fft_size in enumerate(fine_fft_size_list):
            
        fine_fft_size_K = fine_fft_size/1024

        intf_legend = ['']*len(n_sti_list)
        for i_sti, n_sti in enumerate(n_sti_list):
            n_lti = n_lti_list[i_fft,i_sti]
            intf_legend[i_sti] = f'{n_sti=:2d}, {n_lti=:2.0f}'

        fig_text_list1=[[.15,.85,raw_file_stem],
                                [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_all[i_fft,:],precision=0)+ ' MB'],
                                [.15,.79,f'E(SNR) @{f_sigma_drift[-1]*1e-9:.1f} GHz '+ np.array2string(snr_core_db_wavg_all[i_fft,:,-1],precision=1) + ' dB'],
                                [.15,.76,f'Total  '+ np.array2string(time_core_total[i_fft,:],precision=1) + ' sec']]
    
        ext_str = '.png'
        for i_plot in range(n_fig_copies):
            pltg.plot_generic(x_data=[drift_rate_truth_all[:,i_fft,0],drift_rate_core_all[:,i_fft,:]],
                y_data=[ref_snr_db_all[:,i_fft,0],snr_core_db_all[:,i_fft,:]],
                xy_markers = ['-','*'],
                xy_legend = ['Reference',intf_legend],
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[5., np.amax((45.,5.*np.ceil(ref_snr_db/5)+5.))],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + f' Seticore Drifting Tone Detections, Fine FFT Size={fine_fft_size_K:.0f}K  '+test_case,
                fig_text_list=fig_text_list1,
                legend_loc = 'upper right',
                display_fig=display_figs02,
                savfig_name=fig_dir + '11-' + raw_file_stem + f'-FineFFT-{fine_fft_size_K:.0f}K' + '-seticore-det-'+test_case+ext_str)
            fig_text_list1 = [] # do copy of plot without text in figure plot area
            ext_str = 'nt.png'
        


In [ ]:
if isChirp:
    for i_fft, fine_fft_size in enumerate(fine_fft_size_list):
            
        fine_fft_size_K = fine_fft_size/1024

        intf_legend = ['']*len(n_sti_list)
        for i_sti, n_sti in enumerate(n_sti_list):
            n_lti = n_lti_list[i_fft,i_sti]
            intf_legend[i_sti] = f'{n_sti=:2d}, {n_lti=:2.0f}'

        drift_rate_truth = drift_rate_truth_all[:,i_fft,0]

        prob_weight = 30*np.exp(-np.square(np.outer(drift_rate_truth,np.reciprocal(sigma_drift)))/2.)
        prob_legend = ['']*len(sigma_drift)
        for i_cfreq,cfreq in enumerate(f_sigma_drift):
                prob_legend[i_cfreq] = f'Prob Weight {cfreq*1e-9:.1f} GHz'

        fig_text_list1=[[.15,.85,raw_file_stem],
            [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_all[i_fft,:],precision=0)+ ' MB'],
            [.15,.79,f'E(SNR) @{f_sigma_drift[-1]*1e-9:.1f} GHz '+ np.array2string(snr_core_db_wavg_all[i_fft,:,-1],precision=1) + ' dB'],
            [.15,.76,f'Total  '+ np.array2string(time_core_total[i_fft,:],precision=1) + ' sec']]

        ext_str = '.png'
        for i_plot in range(n_fig_copies):
            pltg.plot_generic(x_data=[drift_rate_truth,drift_rate_core_all[:,i_fft,:],drift_rate_truth],
                y_data=[ref_snr_db_all[:,i_fft,0],snr_core_db_all[:,i_fft,:],prob_weight],
                xy_markers = ['-','*','--'],
                xy_legend = ['Reference',intf_legend,prob_legend],
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[0., 50.],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + f' Seticore Drifting Tone Detections, Fine FFT Size={fine_fft_size_K:.0f}K  '+test_case,
                fig_text_list=fig_text_list1,
                legend_loc = 'upper right',
                display_fig=display_figs02,
                savfig_name=fig_dir + '12-' + raw_file_stem + f'-FineFFT-{fine_fft_size_K:.0f}K' + '-seticore-det-'+test_case+ext_str)
            fig_text_list1 = [] # do copy of plot without text in figure plot area
            ext_str = 'nt.png'
         


#### Do plots for selected FFT and n_sti combinations

In [ ]:
if isChirp:
    n_select = len(select_params)
    select_legend = ['']*n_select
    drift_rate_core_select = np.ones((n_det_max,n_select))*np.nan
    snr_core_db_select = np.ones((n_det_max,n_select))*np.nan
    h5_size_MB_select = np.ones((n_select))*np.nan
    time_core_total_select = np.ones((n_select))*np.nan
    snr_core_wavg_db_select = np.ones((n_select))*np.nan
    print(f'{select_params =}')
    print(f'{fine_fft_size_list =}')
    print(f'{n_sti_list =}')
    
    for i_select, s_params in enumerate(select_params):
            
        fine_fft_size = s_params[0]
        n_sti = s_params[1]

        fine_fft_size_K = fine_fft_size/1024
        
        i_fft  = np.where(fine_fft_size_list==fine_fft_size)[0][0]
        i_sti = np.where(n_sti_list==n_sti)[0][0]
        
        n_lti = n_lti_list[i_fft,i_sti]
        select_legend[i_select] = f'{fine_fft_size_K:.0f}K, {n_sti=:2.0f}, {n_lti=:2.0f}'

        try:
            drift_rate_core_select[:,i_select] = drift_rate_core_all[:,i_fft,i_sti]
            snr_core_db_select[:,i_select] = snr_core_db_all[:,i_fft,i_sti]
            h5_size_MB_select[i_select] = h5_size_MB_all[i_fft,i_sti]
            time_core_total_select[i_select] = time_core_total[i_fft,i_sti]
            snr_core_wavg_db_select[i_select] = snr_core_db_wavg_all[i_fft,i_sti,-1]
        except:
            print(f'Select parameters {s_params} out of range')

    prob_weight = 30*np.exp(-np.square(np.outer(drift_rate_truth,np.reciprocal(sigma_drift)))/2.)
    prob_legend = ['']*len(sigma_drift)
    for i_cfreq,cfreq in enumerate(f_sigma_drift):
            prob_legend[i_cfreq] = f'Prob Weight {cfreq*1e-9:.1f} GHz'

    fig_text_list1=[[.15,.85,raw_file_stem],
                [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_select,precision=0)+ ' MB'],
                [.15,.79,f'E(SNR) @{f_sigma_drift[-1]*1e-9:.1f} GHz '+ np.array2string(snr_core_wavg_db_select,precision=1) + ' dB'],
                [.15,.76,f'Total  '+ np.array2string(time_core_total_select,precision=1) + ' sec']]

    ext_str = '.png'
    for i_plot in range(n_fig_copies):
        pltg.plot_generic(x_data=[drift_rate_core_select,drift_rate_truth],
            y_data=[snr_core_db_select,prob_weight],
            xy_markers = ['*','--'],
            xy_legend = [select_legend,prob_legend],
            x_limits=[sig_min_drift-1,sig_max_drift+1],
            y_limits=[0., 40.],
            x_label = 'Drift Rate (Hz/sec)',
            y_label = 'SNR(dB)',
            fig_title= telescope.upper() + f' Seticore Selected Detections  '+test_case,
            fig_text_list=fig_text_list1,
            legend_loc = 'upper right',
            display_fig=display_figs02,
            savfig_name=fig_dir + '14-' + raw_file_stem + f'-FineFFT-{fine_fft_size_K:.0f}K' + '-seticore-det-'+test_case+ext_str)
        fig_text_list1 = [] # do copy of plot without text in figure plot area
        ext_str = 'nt.png'
        
    


#### Do additional plots for selected FFT and n_sti combinations

In [ ]:
if isChirp:
    try:
        n_select = len(select_params2)
    except:
        n_select = 0
    if (n_select>0):
        select_legend = ['']*n_select
        drift_rate_core_select = np.ones((n_det_max,n_select))*np.nan
        snr_core_db_select = np.ones((n_det_max,n_select))*np.nan
        h5_size_MB_select = np.ones((n_select))*np.nan
        time_core_total_select = np.ones((n_select))*np.nan
        snr_core_wavg_db_select = np.ones((n_select))*np.nan
        print(f'{select_params =}')
        print(f'{fine_fft_size_list =}')
        print(f'{n_sti_list =}')
        
        for i_select, s_params in enumerate(select_params2):
                
            fine_fft_size = s_params[0]
            n_sti = s_params[1]

            fine_fft_size_K = fine_fft_size/1024
            
            i_fft  = np.where(fine_fft_size_list==fine_fft_size)[0][0]
            i_sti = np.where(n_sti_list==n_sti)[0][0]
            
            n_lti = n_lti_list[i_fft,i_sti]
            select_legend[i_select] = f'{fine_fft_size_K:.0f}K, {n_sti=:2.0f}, {n_lti=:2.0f}'

            try:
                drift_rate_core_select[:,i_select] = drift_rate_core_all[:,i_fft,i_sti]
                snr_core_db_select[:,i_select] = snr_core_db_all[:,i_fft,i_sti]
                h5_size_MB_select[i_select] = h5_size_MB_all[i_fft,i_sti]
                time_core_total_select[i_select] = time_core_total[i_fft,i_sti]
                snr_core_wavg_db_select[i_select] = snr_core_db_wavg_all[i_fft,i_sti,-1]
            except:
                print(f'Select parameters {s_params} out of range')

        prob_weight = 30*np.exp(-np.square(np.outer(drift_rate_truth,np.reciprocal(sigma_drift)))/2.)
        prob_legend = ['']*len(sigma_drift)
        for i_cfreq,cfreq in enumerate(f_sigma_drift):
            prob_legend[i_cfreq] = f'Prob Weight {cfreq*1e-9:.1f} GHz'

        fig_text_list1=[[.15,.85,raw_file_stem],
                        [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_select,precision=0)+ ' MB'],
                        [.15,.79,f'E(SNR) @{f_sigma_drift[-1]*1e-9:.1f} GHz '+ np.array2string(snr_core_wavg_db_select,precision=1) + ' dB'],
                        [.15,.76,f'Total  '+ np.array2string(time_core_total_select,precision=1) + ' sec']]
    
        ext_str = '.png'
        for i_plot in range(n_fig_copies):
            pltg.plot_generic(x_data=[drift_rate_core_select,drift_rate_truth],
                y_data=[snr_core_db_select,prob_weight],
                xy_markers = ['*','--'],
                xy_legend = [select_legend,prob_legend],
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[0., 40.],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + f' Seticore Selected Detections  '+test_case,
                fig_text_list=fig_text_list1,
                legend_loc = 'upper right',
                display_fig=display_figs02,
                savfig_name=fig_dir + '15-' + raw_file_stem + f'-FineFFT-{fine_fft_size_K:.0f}K' + '-seticore-det-'+test_case+ext_str)
            fig_text_list1 = [] # do copy of plot without text in figure plot area
            ext_str = 'nt.png'

In [ ]:
if isguppi:
    for i_fft, fine_fft_size in enumerate(fine_fft_size_list):
            
        fine_fft_size_K = fine_fft_size/1024

        intf_legend = ['']*len(n_sti_list)
        for i_sti, n_sti in enumerate(n_sti_list):
            n_lti = n_lti_list[i_fft,i_sti]
            intf_legend[i_sti] = f'{n_sti=:2d}, {n_lti=:2.0f}'

        fig_text_list1=[[.15,.85,raw_file_stem],
                [.15,.82,f'{f_min_MHz:.0f}-{f_max_MHz:.0f} MHz, {search_min_drift:.0f}->{search_max_drift:.0f} Hz/sec, {t_obs:.1f} sec'],
                [.15,.79,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_all[i_fft,:],precision=0)+ ' MB'],
                [.15,.76,f'Peak SNR '+ np.array2string(peak_snr_all[i_fft,:],precision=1) + ' dB'],
                [.15,.73,f'Total  '+ np.array2string(time_core_total[i_fft,:],precision=1) + ' sec'],
                [.15,.70,f'n_sti values: '+ np.array2string(n_sti_list,precision=0)]]
        
        ext_str = '.png'
        for i_plot in range(n_fig_copies):
            pltg.plot_generic(x_data=drift_rate_core_all[:,i_fft,:],
                y_data=snr_core_db_all[:,i_fft,:],
                xy_markers = '*',
                xy_legend = intf_legend,
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[5.,40.],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + f'{src_name} Seticore Drifting Tone Detections, Fine FFT Size={fine_fft_size_K:.0f}K',
                fig_text_list=fig_text_list1,
                legend_loc = 'lower right',
                display_fig=display_figs02,
                savfig_name=fig_dir + '16-' + raw_file_stem + f'-FineFFT-{fine_fft_size_K:.0f}K' + '-seticore-det'+test_case+ext_str)
            fig_text_list1 = [] # do copy of plot without text in figure plot area
            ext_str = 'nt.png'


In [ ]:
if isguppi:
    for i_fft, fine_fft_size in enumerate(fine_fft_size_list):
            
        fine_fft_size_K = fine_fft_size/1024

        intf_legend = ['']*len(n_sti_list)
        for i_sti, n_sti in enumerate(n_sti_list):
            n_lti = n_lti_list[i_fft,i_sti]
            intf_legend[i_sti] = f'{n_sti=:2d}, {n_lti=:2.0f}'

        fig_text_list=[[.15,.85,raw_file_stem],
                [.15,.82,f'{f_min_MHz:.0f}-{f_max_MHz:.0f} MHz, {search_min_drift:.0f}->{search_max_drift:.0f} Hz/sec, {t_obs:.1f} sec'],
                [.15,.79,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_all[i_fft,:],precision=0)+ ' MB'],
                [.15,.76,f'Peak SNR '+ np.array2string(peak_snr_all[i_fft,:],precision=1) + ' dB'],
                [.15,.73,f'Total  '+ np.array2string(time_core_total[i_fft,:],precision=1) + ' sec'],
                [.15,.70,f'n_sti values: '+ np.array2string(n_sti_list,precision=0)]]

        ext_str = '.png'
        for i_plot in range(n_fig_copies):
            pltg.plot_generic(x_data=f_start_core_all[:,i_fft,:]*1e-6,
                y_data=snr_core_db_all[:,i_fft,:],
                xy_markers = '*',
                xy_legend = intf_legend,
                x_limits=[f_start1_MHz-0.25,f_stop1_MHz+0.25],
                y_limits=[5.,40.],
                x_label = 'Frequency (MHz)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + f' Seticore Drifting Tone Detections, Fine FFT Size={fine_fft_size_K:.0f}K  '+test_case,
                fig_text_list=fig_text_list1,
                legend_loc = 'lower right',
                display_fig=display_figs02,
                savfig_name=fig_dir + '17-' + raw_file_stem + f'-FineFFT-{fine_fft_size_K:.0f}K' + '-seticore-det-'+test_case+ext_str)
            fig_text_list1 = [] # do copy of plot without text in figure plot area
            ext_str = 'nt.png'


#### Plot composite SNR vs compute time curves

In [ ]:
print(f'sigma_drift = {sigma_drift}')

if isChirp:
    %matplotlib inline
    wavg_max = np.nanmax(snr_core_db_wavg_all)
    wavg_min = np.nanmin(snr_core_db_wavg_all)
    
    # idx = 1     # just choose 3 GHz
    # sigma_drift1 = sigma_drift[idx]
    # if (True):
    for idx, sigma_drift1 in enumerate(sigma_drift):

        f_sigma_drift1 = f_sigma_drift[idx]       
        fine_fft_size_list_K = fine_fft_size_list/1024
    
        ext_str = '.png'
        for i_plot in range(n_fig_copies):
            
            fig = plt.figure(figsize=(10, 6))
            
            for i_fft, fine_fft_size_K in enumerate(fine_fft_size_list_K):
                plt.plot(time_core_total[i_fft,:],snr_core_db_wavg_all[i_fft,:,idx],'-*',label=f'FFT={fine_fft_size_K:.0f}K')
            
            # plt.xlim(0.,4.*np.ceil(np.nanmax(time_core_total[i_fft,:]/4.)))
            # plt.xlim(2.*np.floor(np.nanmin(time_core_total[0,:]/2.)),2.*np.ceil(np.nanmax(time_core_total[i_fft,:]/2.)))
            xlim_max = 2.*np.ceil(np.nanmax(time_core_total[:]/2.))
            if (xlim_max<10.): xlim_max=10.;
            plt.xlim(0.,xlim_max)
        
            plt.title(f'{telescope.upper()} Seticore Drifting Tone Detections, SNR vs. Compute Time at {f_sigma_drift1*1e-9:.1f}GHz  '+test_case )

            # plt.ylim(5*(np.floor((wavg_min-2)/5)),5*(np.ceil(wavg_max/5)+1))
            plt.ylim(10.,40.)
            plt.xlabel('Compute Time Per Coarse Channel sec')
            plt.ylabel('SNR Expected Value dB')
            if i_plot==0:
                plt.figtext(.15,.85,raw_file_stem)
                plt.figtext(.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_all[0,:],precision=0)+ ' MB')
                plt.figtext(.15,.79,f'n_sti values: '+ np.array2string(n_sti_list,precision=0))
            plt.legend(loc='upper right')
            plt.grid()

            fig_name = fig_dir + '21-' + raw_file_stem + f'-fctr-{f_sigma_drift1*1e-9:.0f}GHz' + '-seticore-snr-time-'+test_case+ext_str
                        
            print(fig_name)
            plt.savefig(fig_name,bbox_inches='tight')

            if display_figs02:
                plt.show()
            else:
                plt.close(fig)
                
            ext_str = 'nt.png' # for second pass, do copy of plot without text in figure plot area
        

In [ ]:
if isguppi:
    %matplotlib inline
    fine_fft_size_list_K = fine_fft_size_list/1024

    ext_str = '.png'
    for i_plot in range(n_fig_copies):    
    
        fig = plt.figure(figsize=(10, 6))
        
        for i_fft, fine_fft_size_K in enumerate(fine_fft_size_list_K):
            plt.plot(time_core_total[i_fft,:],peak_snr_all[i_fft,:],'-*',label=f'FFT={fine_fft_size_K:.0f}K')
                
        plt.title(f'{src_name} {telescope.upper()} Seticore Drifting Tone Detections, SNR vs. Compute Time  '+test_case)

        plt.xlim(0.,5.*np.ceil(np.amax(time_core_total[i_fft,:]/5.)))
        plt.ylim(5.,40.)
        plt.xlabel('Compute Time sec')
        plt.ylabel('SNR Expected Value dB')
        if i_plot==0:
            plt.figtext(.15,.85,raw_file_stem)
            plt.figtext(.15,.82,f'{f_min_MHz:.0f}-{f_max_MHz:.0f} MHz, {search_min_drift:.0f}->{search_max_drift:.0f} Hz/sec, {t_obs:.1f} sec')
            plt.figtext(.15,.79,f'raw {raw_size_MB:3.0f} MB, h5 '+np.array2string(h5_size_MB_all[0,:],precision=0)+ ' MB')
            plt.figtext(.15,.76,f'n_sti values: '+ np.array2string(n_sti_list,precision=0))
        plt.legend(loc='upper right')
        plt.grid()

        fig_name = fig_dir + '25-' + raw_file_stem  + '-seticore-snr-time-'+test_case+ext_str
        print(fig_name)
        plt.savefig(fig_name,bbox_inches='tight')

        if display_figs02:
            plt.show()
        else:
            plt.close(fig)
        ext_str = 'nt.png' # for second pass, do copy of plot without text in figure plot area
 

In [ ]:
print('Done')
print('Execution times in seconds:')
print(f'{time_rawspec = }')
if run_turbo:
    print(f'{time_turbo = }')
    print(f'{time_turbo_total = }')
    
print(f'{time_core = }')
print(f'{time_core_total = }')

print(f'{time_run = }')

# Beep in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")
os.system("powershell.exe '[console]::beep(261.6,700)'")

